<a href="https://colab.research.google.com/github/jakezetci/sun/blob/main/analysis_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импортируем всё необходимое


---



In [ ]:
import numpy as np
import pickle
import pandas as pd
import time
from google.colab import drive
!pip install astropy[all]
!pip install sunpy[all]
!pip install pyTelegramBotAPI



In [ ]:
!git clone https://github.com/jakezetci/sun.git


In [ ]:
!cd

try:
  from sun.pipeline import download_map_and_harp, bitmaps_to_points, compute_harp_MEnergy
  from sun.computing import compute_grid_energy, create_grid, comp_grid_points
  from sun.plots import config
except ModuleNotFoundError:
  from pipeline import download_map_and_harp, bitmaps_to_points, compute_harp_MEnergy
  from computing import compute_grid_energy, create_grid, comp_grid_points
  from plots import config

# Следующее: подсчёт магнитных энергий в определенный момент времени.

In [ ]:
M = 30 # количество временных точек
frequency = '6H' # частота точек
start_time = "2023-08-08 00:12:00" # время начала измерений

#параметры сетки, которую мы будем считать
latlim = (-60, 60)
lonlim = (-60, 60)
N = 1 # количество точек на один градус
r = 800000 * 1e3 # высота сетки в метрах

In [ ]:
dates = pd.date_range(start=strat_time,
                      freq=frequency, periods=M).values
energys_low = np.zeros(M)
energys_high = np.zeros(M)
for i, date in enumerate(dates):
    tic=time.perf_counter()
    magnetogram, bitmaps = download_map_and_harp(date, date)
    energy_low = np.sum(compute_harp_MEnergy(date, date, downloaded=True,
                                             magnetogram=magnetogram,
                                             bitmaps=bitmaps))
    values, points = bitmaps_to_points(date, downloaded=True, magnetogram=magnetogram,
                                       bitmaps=bitmaps)
    grid_empty = create_grid(latlim, lonlim, N, r=r)
    ts = pd.to_datetime(str(date))
    name = ts.strftime('%Y.%m.%d %I:%M%p')
    grid_high = comp_grid_points(grid_empty, points, values,
                                 checkpoints=15, timestamp=True,
                                 alert=False, name=name)
    energy_high = compute_grid_energy(grid_high)
    energys_low[i] = energy_low
    energys_high[i] = energy_high
    toc=time.perf.counter()
    print(f'one value done in  {toc - tic:0.2f} seconds')
# сохранение результатов
drive.mount('/content/gdrive')
np.savetxt('/content/gdrive/My Drive/HIGH.txt', energys_high)
np.savetxt('/content/gdrive/My Drive/LOW.txt', energys_low)
np.savetxt('/content/gdrive/My Drive/dates.txt', dates)